ライブラリをインポートする

In [ ]:
!python -m pip install semantic-kernel==0.5.1.dev0
!python -m pip install slack-sdk load_dotenv
!python -m pip install guidance

コンフィグを読み取る

In [ ]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    OpenAIChatCompletion
)

kernel = sk.Kernel()

useAzureOpenAI = True

if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    azure_chat_service = AzureChatCompletion(
        deployment_name=deployment,
        api_key=api_key,
        endpoint=endpoint
    )
    kernel.add_chat_service("chat_completion", azure_chat_service)
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    openai_chat_service = OpenAIChatCompletion(
        ai_model_id="gpt-3.5-turbo",
        api_key=api_key,
        org_id=org_id
    )
    kernel.add_chat_service("chat-gpt", openai_chat_service)

ユーザとやりとりをして、Planを作成する

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

import guidance
from guidance import system, user, assistant, gen

buddy = guidance.models.AzureOpenAIChat(
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

skills = [
    {"name": "SendMessage", "description": "Send a message to a Slack channel."},
    {"name": "GenerateMessageJa", "description": "Generate a message in Japanese."}
]

@guidance
def experts(lm, input, skills):
    with system():
        lm += f"""\
        # 前提条件
            - あなたはユーザの入力に基づいて「実行計画」を作成する優秀なプランナーです。
            - 実行計画はユーザの入力を達成するために必要な作業内容を分解および順序づけしたものです。
            - 実行計画には関数の利用を含みます。
            - 実行計画で利用できる関数は「使用可能な関数」で定義された関数のみです。
        """

    with user():
        lm += f"""
        # ゴール
            - ユーザの入力から実行計画を作成する
            - 出力形式は以下のようなJSONフォーマットであること
                ```
                {{
                    "user_input": "ユーザの入力",
                    "best_plan": [実行する関数名のリスト],
                    "description": "実行する関数名とその関数を利用する目的の説明"
                }}
                ```
        # 実行のプロセス
            1. ユーザの入力を受け取り、実行計画を3つ作成する
            2. 最も有効な実行計画が1位となるようにランク付けする
            3. 1位の実行計画をJSONフォーマットに変換する
            4. JSONフォーマットに変数した実行計画のみを出力する
        # 使用可能な関数
        """

        for i in range(len(skills)):
            lm += f"""
                - {skills[i]["name"]}: {skills[i]["description"]}
            """

    with assistant():
        lm += f"""\
            はい。ユーザの入力に基づいて実行計画を作成し、JSONのみを出力します。
            また、実行計画の作成には使用可能な関数のみを使用します。
        """
    
    with user():
        lm += f"""\
            ユーザの入力:{input}
            必ずJSON形式の部分のみを出力してください。
        """
    
    with assistant():
        lm += gen(name='answer', max_tokens=500)

    return lm

result = None

while True:
    user_input = input("なにを依頼したいですか？特に依頼がなければ'end'と入力してください。:")
    if user_input == "end":
        break
    else:
        result = buddy + experts(input=user_input, skills=skills)

In [ ]:
print(result['answer'])

SKにPlanを作成させる

In [ ]:
from plugins.SlackPlugin.slack_plugin import SlackPlugin
from semantic_kernel.planning.basic_planner import BasicPlanner

plugins_directory = "plugins"
messageFunctions = kernel.import_semantic_plugin_from_directory(plugins_directory, "MessagePlugin")

slack_plugin = kernel.import_plugin(SlackPlugin(), "SlackPlugin")

plan_base = result["answer"]

print(f"### Plan ###\n{plan_base}")

basic_planner = BasicPlanner()
basic_plan = await basic_planner.create_plan(plan_base, kernel)

print(basic_plan.generated_plan)

In [ ]:
results = await basic_planner.execute_plan(basic_plan, kernel)
print(results)